In [7]:
!python3.11 -m pip install pymystem3

In [2]:
import pandas as pd
import string
import nltk
import warnings

from pymystem3 import Mystem
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import sys
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.optim import Adam
import os
import subprocess

from matplotlib import pyplot as plt
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import shutil
import random
import itertools
from datasets import load_metric

bleu_metric = load_metric("bleu")

/var/folders/64/q9vthv094_591jd1jlz91_x00000gn/T/ipykernel_22831/1375413025.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")


In [3]:
VALIDATION_PROP = 0.1
RANDOM_STATE = 79

X_TRAIN_PATH = 'en_train.txt'
Y_TRAIN_PATH = 'ru_train.txt'
X_VALID_PATH = 'en_valid.txt'
Y_VALID_PATH = 'ru_valid.txt'
TRAIN = False

HIDDEN_SIZE = 256
LSTM_LAYERS = 3
INPUT_SIZE = 256
BATCH_SIZE = 32
IS_BIDIRECTIONAL_LAYERS = False
CNT_CLASSES = None

SOS_TOKEN = "<SOS>"
EOS_TOKEN = "<EOS>"
UNK_TOKEN = "<UNK>"
PAD_TOKEN = "<PAD>"
PAD_IND = 3
SOS_IND = 0
EOS_IND = 1
EPOCHS = 10
MODEL_NAME = 'lstm_basic_model'

SCHEDULER_LAMBDA_PARAM = 0.95

device = 'cuda' if torch.cuda.is_available() else 'cpu'

MODEL_FOLDER_PATH = MODEL_NAME
LR = 0.0004
CLIP = 1
TEACHER_FORCING_RATIO = 0

WRITE_CNT_STEPS = 100
SCHEDULER_CNT_STEPS = 300
GREEDY_DECODING_LABEL = 'greedy'
BEAM_SEARCH_DECODING_LABEL = 'beam_search'

NEC_TOKENS_LEN = 100
MAX_TEXT_LEN = 100
BEAM_WIDTH = 3

signs = '.,?!:; '
numbers = ''.join([str(i) for i in range(10)])
RUSSIAN_LETTERS_AND_SIGNS = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + signs + numbers
ENGLISH_LETTERS_AND_SIGNS = ''.join([chr(i) for i in range(ord('a'), ord('z') + 1)]) + signs + numbers
print(RUSSIAN_LETTERS_AND_SIGNS, ENGLISH_LETTERS_AND_SIGNS)

абвгдеёжзийклмнопрстуфхцчшщъыьэюя.,?!:; 0123456789 abcdefghijklmnopqrstuvwxyz.,?!:; 0123456789


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/d1ffic00lt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
warnings.filterwarnings("ignore")

In [6]:
with open("train/train_stt/0.txt") as file:
    test_data = file.read()

In [7]:
test_data

'[0.00s -> 6.34s]  Давай поспорим на выпуск, вот типа, что я точно угадаю цену одного из предметов.\n[6.34s -> 7.34s]  Ох, зима.\n[7.34s -> 9.34s]  Доктор Клим Докностраджа.\n[9.34s -> 11.34s]  Мамочка, прости меня, пожалуйста, я негодяй.\n[12.34s -> 16.34s]  Блин, клянула у вас в специальной связи, я вам сразу скажу.\n[16.34s -> 17.34s]  Открывай рот.\n[34.19s -> 37.39s]  Спермокряха для осеменения стоит полторы тысячи рублей.\n[37.93s -> 40.93s]  Шарма Димы Масленникова Эль Классико стоит двести рублей,\n[40.93s -> 44.93s]  а синий-синий Ламборгини Рахима стоит двадцать пять миллионов рублей.\n[44.93s -> 46.93s]  Вы спросите, откуда я это знаю?\n[46.93s -> 48.93s]  Все просто, у всего есть своя цена.\n[48.93s -> 52.93s]  Меня зовут Макс Клим Док, и это шоу «Правильная цена».\n[54.93s -> 55.93s]  Молодцы, молодцы.\n[55.93s -> 58.93s]  Давайте познакомимся с нашими участницами.\n[58.93s -> 59.93s]  И участниками.\n[59.93s -> 63.93s]  Первая девушка – участница SkyFam BinetSense.\n[63.9

In [8]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")
mystem = Mystem()

def process_text(text):
    texts = [i.split("]  ")[1].strip().lower() for i in text.split("\n")]
    for text in range(len(texts)):
        result = ""
        for letter in texts[text]:
            if letter.isalnum():
                result += letter
            elif letter == " ":
                result += " "

        texts[text] = result

    texts = mystem.lemmatize(" ".join(texts))
    tokens = [i for i in texts if i not in russian_stop_words]
    return " ".join(word_tokenize("".join(tokens).strip()))

In [9]:
process_text(test_data)

'давать поспорить выпуск тип точно угадывать цена предмет ох зима доктор клим докностраджа мамочка прощать пожалуйста негодяй блин клядывать специальный связь сразу сказать открывать рот спермокряха осеменение стоить полтора тысяча рубль шарм дима масленников эль классико стоить двести рубль синийсиний ламборгини рахима стоить двадцать пять миллион рубль спрашивать откуда это знать весь просто весь свой цена звать макс клим док шоу правильный цена молодец молодец давать познакомиться наш участница участник первый девушка участница skyfam binetsense это второй участник full house память хороший игрок fifu женя лизогубенко участница exo team наш игра очень простой сыграть раунд время раунд купить балл участник оказываться близко правильный цена получать балл второй место балл оказываться далеко весь ноль балл конец набирать большой количество балл участник выбирать подарок давать немножко разогнаться каждый задавать вопрос binet самый дорогой чек закрывать ресторан ресторан самый дорогой

In [ ]:
def clean_text(text):
    text = text.lower().strip()
    res_text = ''.join([
        tt for tt in text if tt in RUSSIAN_LETTERS_AND_SIGNS or tt in ENGLISH_LETTERS_AND_SIGNS
    ])
    return res_text

In [10]:
class Tokenizer(object):
    def __init__(self, texts, sos, eos, unk, pad, tokens=None):
        self.sos = sos
        self.eos = eos
        self.unk = unk
        self.pad = pad
        self.tokens = tokens

        self.create_tokenizer(texts)

    def create_tokenizer(self, texts):
        texts = [clean_text(text) for text in texts] # clean each text

        tokens = set('\n'.join(list(texts)).replace('\n', '')) # get tokens (symbols)
        tokens = [t for t in tokens if t not in (self.sos, self.eos, self.unk) and len(t)]
        self.tokens = self.tokens or [self.sos, self.eos, self.unk, self.pad] + tokens

        self.token2idx = {token: i for i, token in enumerate(self.tokens)}
        self.idx2token = {i: token for i, token in enumerate(self.tokens)}
        self.unk_idx = self.token2idx[self.unk]
        self.sos_idx = self.token2idx[self.sos]
        self.eos_idx = self.token2idx[self.eos]
        self.pad_idx = self.token2idx[self.pad]

    def tokenize(self, text):
        #converts text to a list of tokens
        tokens = []
        for tok in text:
            if tok in self.token2idx:
                tokens.append(tok)
            else:
                tokens.append(self.unk_idx)

        return [self.sos] + tokens + [self.eos]

    def convert_tokens_to_idx(self, tokens):
        idx_list = [self.token2idx.get(tok, self.unk_idx) for tok in tokens]

        return idx_list

    def convert_text_to_idx(self, text, seq_len=None):
        tokens = self.tokenize(text)[:seq_len]

        return self.convert_tokens_to_idx(
            tokens
        )

    def convert_idx_to_tokens(self, idx_list):
        ans = []
        for idx in idx_list:
            ans.append(self.idx2token[idx])

        return ans

In [11]:
def prepare_tokenizer(texts_path):
    with open(texts_path) as f:
        lines = [clean_text(line.strip()) for line in f.readlines()]

    return Tokenizer(lines, sos=SOS_TOKEN, eos=EOS_TOKEN, unk=UNK_TOKEN, pad=PAD_TOKEN)

In [ ]:
class TextDataset(Dataset):
    def __init__(self,
                 x_seq_path,
                 x_tokenizer=None,
                 x_seq_len=None,
                 is_train=True,
                 y_seq_path=None,
                 y_tokenizer=None,
                 y_seq_len=None):
        self.is_train = is_train
        with open(x_seq_path) as f:
            self.x_seq_list = [line.strip() for line in f.readlines()]

        self.x_tokenizer = x_tokenizer or prepare_tokenizer(x_seq_path)
        self.x_seq_len = x_seq_len

        if self.is_train:
            with open(y_seq_path) as f:
                self.y_seq_list = [line.strip() for line in f.readlines()]
            self.y_tokenizer = y_tokenizer or prepare_tokenizer(y_seq_path)
        else:
            self.y_seq_list = None
            self.y_tokenizer = None
        self.y_seq_len = y_seq_len

    def __getitem__(self, idx):

        x = clean_text(self.x_seq_list[idx])
        x = self.x_tokenizer.convert_text_to_idx(x, seq_len=self.x_seq_len)
        x_pad_part = [self.x_tokenizer.pad_idx] * max(0, self.x_seq_len - len(x))
        x = x + x_pad_part

        x = torch.tensor(x)
        if self.is_train:
            y = clean_text(self.y_seq_list[idx])
            y = self.y_tokenizer.convert_text_to_idx(y, seq_len=self.y_seq_len)[1:]
            y_pad_part = [self.y_tokenizer.pad_idx] * max(0, self.y_seq_len - len(y))
            y = y + y_pad_part
            y = torch.tensor(y)

            return x, y
        return x

    def cnt_y_tokens(self):
        return len(self.y_tokenizer.tokens)

    def cnt_x_tokens(self):
        return len(self.x_tokenizer.tokens)

    def __len__(self):
        return len(self.x_seq_list)

In [ ]:
class EncoderDecoderModel(nn.Module):
    # Simple Encoder-Decoder Model with 3 GRU Layers in encoder
    # And with 3 GRU in decoder, forward could be run with teacher_forcing
    def __init__(self,
                 input_voc_size,
                 cnt_classes,
                 device=None,
                 input_size=INPUT_SIZE,
                 hidden_size=HIDDEN_SIZE,
                 lstm_layers=LSTM_LAYERS,
                 is_bidirectional=IS_BIDIRECTIONAL_LAYERS):
        super(EncoderDecoderModel, self).__init__()

        self.input_size = input_size
        self.lstm_layers = lstm_layers
        self.cnt_classes = cnt_classes
        self.hidden_size = hidden_size
        self.out_features = 1 if self.cnt_classes == 2 else self.cnt_classes
        self.is_bidirectional = is_bidirectional

        self.encoder_embed = nn.Embedding(input_voc_size, self.input_size)
        self.encoder_lstm_layer = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.lstm_layers,
            bidirectional=self.is_bidirectional,
            batch_first=True
        )

        self.decoder_embed = nn.Embedding(self.cnt_classes, self.input_size)
        self.decoder_lstm_layer = nn.LSTM(
            input_size=self.hidden_size,
            hidden_size=(1 + self.is_bidirectional) * self.hidden_size,
            num_layers=self.lstm_layers,
            batch_first=True
        )
        self.decoder_linear = nn.Linear(
            in_features=self.hidden_size,
            out_features=self.out_features)

    def forward(
            self,
            x,
            target_val,
            teacher_forcing_ratio = TEACHER_FORCING_RATIO):
        batch_n = x.shape[0]
        length_tensor = (x != PAD_IND).sum(axis=1).cpu() # get seq lens to pack

        # encoder
        x = self.encoder_embed(x)
        x = pack_padded_sequence(x, length_tensor, batch_first=True, enforce_sorted=False)

        _, prev_state = self.encoder_lstm_layer(x)

        # reshape prev_state
        prev_state = (prev_state[0].reshape(self.lstm_layers, batch_n, -1),
                      prev_state[1].reshape(self.lstm_layers, batch_n, -1))

        # create <sos> tokens for each sentence
        prev_token_idx = torch.tensor([SOS_IND] * batch_n).reshape(-1, 1).to(device)

        # create probs for each token
        probs_tensor = None
        for i in range(NEC_TOKENS_LEN):
            if i != 0 and target_val is not None and random.random() < teacher_forcing_ratio:
                # teacher forcing
                prev_token_idx = target_val[:, i - 1].reshape(-1, 1)

            # decoder step
            prev_token_emb = self.decoder_embed(prev_token_idx)
            out, prev_state = self.decoder_lstm_layer(prev_token_emb, prev_state)

            out = self.decoder_linear(out)

            # token idx answer by greedy strategy
            prev_token_idx = out.argmax(axis=-1).reshape(batch_n, -1)

            # save probs for answers
            if probs_tensor is None:
                probs_tensor = out.unsqueeze(0)
            else:
                probs_tensor = torch.cat([probs_tensor, out.unsqueeze(0)])

            #if prev_token_idx.unique().shape[0] == 1 and prev_token_idx[0] == EOS_IND:
            #break

        return probs_tensor.transpose(1, 0)

In [ ]:
# create train and val dataset (val tokenizers are the same as in train)
train_dataset = TranslationDataset(
    x_seq_path=X_TRAIN_PATH,
    y_seq_path=Y_TRAIN_PATH,
    is_train=True,
    x_seq_len=NEC_TOKENS_LEN,
    y_seq_len=NEC_TOKENS_LEN)
val_dataset = TranslationDataset(
    x_seq_path=X_VALID_PATH,
    x_tokenizer=train_dataset.x_tokenizer,
    y_seq_path=Y_VALID_PATH,
    x_seq_len=NEC_TOKENS_LEN,
    y_seq_len=NEC_TOKENS_LEN,
    y_tokenizer=train_dataset.y_tokenizer,
    is_train=True
)

# create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
# cnt classes if it is neccessary
if CNT_CLASSES is None:
    CNT_CLASSES = train_dataset.cnt_y_tokens()

In [ ]:
# define model and move it to device
model = EncoderDecoderModel(
    input_voc_size=train_dataset.cnt_x_tokens(),
    device=device,
    cnt_classes=train_dataset.cnt_y_tokens())
model = model.to(device)

In [ ]:
# use CrossEntropyLoss because the task is something like classification
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=train_dataset.x_tokenizer.pad_idx)
# define optimizer
optimizer = Adam(model.parameters(), lr=LR)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# define writer for using tensorboard
writer = SummaryWriter('runs/{}'.format(MODEL_NAME))

# define scheduler (policy of lr change)
lambda_scheduler = lambda x: SCHEDULER_LAMBDA_PARAM ** x
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_scheduler)

